In [38]:
# Basics
import pandas as pd
import numpy as np

# Text pre-processing
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.stem import WordNetLemmatizer 
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from gensim.sklearn_api import W2VTransformer
# Models
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import StratifiedKFold

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [16]:
df = pd.read_pickle('./data/labeled/final_annotated_data_roberta.pickle')
df.head()

,id,comment,parent_comment,original_post,divisiveness,divisiveness_keyword,human_incivility,stereotype,namecalling,aspersion,...,third_party_attack,civility,comment_de,comment_en,bert_aug,attention_aug,roberta,attention_mask,roberta_aug,attention_mask_aug
0,fysijl2,"If I wanted to watch a stable genius, I'd rath...",It's going to devolve into a campaign stump sp...,Today President Trump is expected to hold his ...,1,"Trump, coronavirus",2,0,3,0,...,1,4,"Wenn ich ein stabiles Genie sehen wollte, würd...","If I wanted to see a stable genius, I'd rather...","[101, 2065, 1045, 2359, 2000, 2156, 1037, 6540...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1106, 38, 770, 7, 1183, 10, 4375, 16333, 6...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1106, 38, 770, 7, 192, 10, 4375, 16333, 6,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,fygrqc9,If they're willing to shoot a deceptive video ...,"Look, you're simply wrong or haven't done the ...",Media are just plain lying about Kayleigh McEn...,1,Kayleigh McEnany,0,0,0,0,...,0,1,"Wenn sie bereit sind, ein irreführendes Video ...",If they are willing to make a misleading video...,"[101, 2065, 2027, 2024, 5627, 2000, 2191, 1037...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1106, 51, 214, 2882, 7, 4511, 10, 31405, 5...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1106, 51, 32, 2882, 7, 146, 10, 12030, 569...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,g0ql4qq,Would be thrilling if he ended up actually bei...,Second challenge is disputing just about every...,Today President Trump is expected to hold his ...,1,Kanye West,1,0,0,0,...,0,1,"Wäre spannend, wenn er am Ende tatsächlich für...",It would be exciting if he was actually punish...,"[101, 2009, 2052, 2022, 10990, 2065, 2002, 200...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 29042, 28, 16208, 114, 37, 1249, 62, 888, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 243, 74, 28, 3571, 114, 37, 21, 888, 14459...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,fz1nlup,Nope they like to let one vote side against th...,LETS GOOOO. Romney found his balls!!,Romney reportedly will vote against controvers...,1,"Romney, controversial Fed nominee",0,0,0,0,...,0,0,"Nein, sie lassen gerne eine Seite gegen die Di...","No, they like to let one side vote against thi...","[101, 2053, 1010, 2027, 2066, 2000, 2292, 2028...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 487, 9877, 51, 101, 7, 905, 65, 900, 526, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 3084, 6, 51, 101, 7, 905, 65, 526, 900, 13...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,fzd1ea2,"Most conservatives agree with you. However, co...",you know you can commit a violent crime and no...,If african americans commit 3x more violent cr...,1,"african american crime, police",2,1,0,1,...,0,3,Die meisten Konservativen stimmen Ihnen zu. Ab...,"Most conservatives agree with you, but to comm...","[101, 2087, 11992, 5993, 2007, 2017, 1010, 202...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 2895, 9930, 2854, 19, 47, 4, 635, 6, 11389...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 2895, 9930, 2854, 19, 47, 6, 53, 7, 6225, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [35]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=2,
                        ngram_range=(1, 2), 
                        stop_words='english')

corpus = []
corpus_string = []
lemmatizer = WordNetLemmatizer() 
for t in  list(df['comment']):
    corpus_string.append(t)
    tokens_raw = simple_preprocess(t)
    tokens_lemm = []
    for token in tokens_raw:
        tokens_lemm.append(lemmatizer.lemmatize(token))
    corpus.append(tokens_lemm)
w2v_model = Word2Vec(corpus, min_count=1)

features = tfidf.fit_transform(df.comment).toarray()

count = CountVectorizer(max_df=0.8, min_df=2, stop_words='english')

In [39]:
def sentence_vector(tokens):
    sums = np.zeros(100)
    for word in tokens:
        new_word = lemmatizer.lemmatize(word)
        sums = sums + np.array(w2v_model[new_word])
    return sums

def extract_w2v_features(training_data, testing_data):
    X_train = []
    X_test = []
    for sentence in training_data['comment']:
        vec = sentence_vector(simple_preprocess(sentence))
        X_train.append(vec)
    for sentence in testing_data['comment']:
        vec = sentence_vector(simple_preprocess(sentence))
        X_test.append(vec)
    return X_train, X_test

In [36]:
from sklearn.metrics import f1_score, precision_score, roc_auc_score, recall_score
def flat_accuracy(pred_flat, labels_flat):
    try:
        roc = roc_auc_score(pred_flat, labels_flat)
    except ValueError:
        roc = 0
    return f1_score(pred_flat, labels_flat, average='macro'), precision_score(pred_flat, labels_flat, average='macro'), recall_score(pred_flat, labels_flat, average='macro'), roc, np.sum(pred_flat == labels_flat) / len(labels_flat)

In [40]:
tfidfPipeline = Pipeline(steps=[('vect',tfidf), ('clasfi', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=-1))])
countPipeline = Pipeline(steps=[('vect',count), ('clasfi', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=-1))])
word2vecPipeline = Pipeline(steps=[('clasfi', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=-1))])

pipelines = { 'tfidf': tfidfPipeline, 'count': countPipeline, 'word2vec': word2vecPipeline }
training_stats = []

categories = ['aspersion', 'civility', 'vulgarity', 'personal_attack', 'third_party_attack', 'stereotype']
# categories = ['aspersion']
for pname, p in pipelines.items(): 
    for c in categories:
        print(f'Processing **{c}** comments...')
        
        total_folds = 5
        current_fold = 0
        fold=StratifiedKFold(n_splits=total_folds, shuffle=True, random_state=1000)

        for train_index, test_index in fold.split(df, df[c]):
            current_fold += 1
            train, test = df.iloc[train_index], df.iloc[test_index]
            
            if pname == 'word2vec':
                X_train, X_test = extract_w2v_features(train, test)
            else:
                X_train, X_test = train['comment'], test['comment']
            p.fit(X_train, train[c])
            prediction = p.predict(X_test)
            f1, precision, recall, auc, accuracy = flat_accuracy(test[c].astype(bool).astype(int), prediction)
            
            training_stats.append(
                {
                    'category': c,
                    'Valid. Accur.': accuracy,
                    'f1_score' : f1,
                    'recall_score': recall,
                    'precision_score': precision,
                    'AUC_score': auc,
                    'fold' : current_fold,
                    'model': pname
                }
            )

Processing **aspersion** comments...
Processing **civility** comments...


/Users/yujia/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/yujia/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


Processing **vulgarity** comments...
Processing **personal_attack** comments...
Processing **third_party_attack** comments...
Processing **stereotype** comments...
Processing **aspersion** comments...


/Users/yujia/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/yujia/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/yujia/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/yujia/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/yujia/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:

Processing **civility** comments...


...

/Users/yujia/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/yujia/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/yujia/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/yujia/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/yujia/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:

Processing **third_party_attack** comments...


/Users/yujia/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/yujia/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/yujia/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/yujia/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/yujia/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:

Processing **stereotype** comments...


/Users/yujia/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/yujia/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/yujia/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/yujia/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/yujia/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:

In [41]:
df_stats = pd.DataFrame(data=training_stats)
df_stats

,category,Valid. Accur.,f1_score,recall_score,precision_score,AUC_score,fold,model
0,aspersion,0.900990,0.473958,0.5,0.450495,0.5,1,tfidf
1,aspersion,0.900990,0.473958,0.5,0.450495,0.5,2,tfidf
2,aspersion,0.900990,0.473958,0.5,0.450495,0.5,3,tfidf
3,aspersion,0.899340,0.473501,0.5,0.449670,0.5,4,tfidf
4,aspersion,0.899340,0.473501,0.5,0.449670,0.5,5,tfidf
...,...,...,...,...,...,...,...,...
85,stereotype,0.948845,0.486876,0.5,0.474422,0.5,1,word2vec
86,stereotype,0.948845,0.486876,0.5,0.474422,0.5,2,word2vec
87,stereotype,0.948845,0.486876,0.5,0.474422,0.5,3,word2vec
88,stereotype,0.947195,0.486441,0.5,0.473597,0.5,4,word2vec


In [42]:
df_stats.groupby('model').mean()

,Valid. Accur.,f1_score,recall_score,precision_score,AUC_score,fold
model,,,,,,
count,0.824202,0.477598,0.479839,0.522881,0.547967,3.0
tfidf,0.833993,0.450182,0.488554,0.495236,0.505458,3.0
word2vec,0.832563,0.452094,0.500000,0.416282,0.500000,3.0
